# Aggregate the OGGM raw files to regional or global summed up csv files


*Note that the paths of this notebook only work if you are running the notebook from the OGGM cluster*


In [1]:
import xarray as xr
import pandas as pd
import progressbar
import numpy as np
from oggm.utils import mkdir
import matplotlib.pyplot as plt
import json
import os

In [2]:
%pwd

'/home/www/oggm/oggm-standard-projections/oggm-standard-projections-csv-files/notebooks'

In [3]:
dirpath = '/home/www/oggm/oggm-standard-projections/oggm_v16/2023.3/'
outputpath = '/home/www/oggm/oggm-standard-projections/oggm-standard-projections-csv-files/1.6.1'
# we only aggregate here `w5e5_gcm_merged`, but you could repeat this here with `gcm_from_2000` as it is also available in the raw per-glacier datafiles
historical_future_option = 'w5e5_gcm_merged'

# this is the only preprocessed version (i.e., we match the geodetic observations on any individual glacier)
exp = 'match_geod_pergla'
oggm_version = 'oggm_v1.6.1_2023.3'

rgi_meta = pd.read_hdf('/home/www/oggm/rgi/rgi62_stats.h5')
rgi_meta = rgi_meta.loc[rgi_meta.Connect != 2]

## A. Common running glacier aggregations for all projections, until 2100 and until 2300, i.e. `common_running_2100_2300`
- the same for only until 2100 is done in Section B further below

### A. 1: GCMs - missing glaciers overview  

In [4]:
allfiles = []
for root, dirs, files in os.walk(dirpath):
    for file in files:
        if (file.endswith(".nc")) and ('run_hydro' in file) and (historical_future_option in file) and ('_0_1000' in file) and ('basin' not in file):
             allfiles.append(os.path.join(root, file))

df_meta = pd.DataFrame()
invalid_per_reg = {}  
meta_per_reg = {}

In [5]:
run = False
if run:
    # this takes very long !!!
    invalid_per_reg = {}  
    meta_per_reg = {}
    for f in allfiles: #progressbar.progressbar(allfiles):
        ename = f.replace(dirpath, '')
        ss = ename.split('/')
        cmip = ss[0]
        endyr = ss[1]
        rgi_reg = ss[2]
        gcm_original = ss[3].split('_')[5] #ss[2]
        gcm = gcm_original.upper()
        if 'ISIMIP3b' in cmip:
            ssp = ss[3].split('_')[7]
            bc = 'ISIMIP3b:1979-2014'
        else:
            ssp = ss[3].split('_')[6]
            bc = '2000-2019'

        end0,end1 = ss[3].split('_')[-2:]
        end = end0+'_'+end1
        ff = f[:-len(end)]+'*'

        #if ssp not in ['ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp585']:
        #    continue

        # exp is the same everywhere, so we don't need as an id... 
        run_id = f'{cmip}:{endyr}:{gcm}:{ssp}:{rgi_reg}'
        df_meta.loc[run_id, 'cmip'] = cmip
        df_meta.loc[run_id, 'gcm_original'] = gcm_original # some have lower and upper cases in the raw output files
        df_meta.loc[run_id, 'gcm'] = gcm  # for the aggregated files, all GCMs are in upper cases
        df_meta.loc[run_id, 'scenario'] = ssp
        df_meta.loc[run_id, 'rgi_reg'] = rgi_reg
        df_meta.loc[run_id, 'end_year'] = 0
        df_meta.loc[run_id, 'bias_correction'] = bc

        df_meta.loc[run_id, 'perc_area_missing'] = 0

        df_meta.loc[run_id, 'fpath'] = ff

        if rgi_reg not in invalid_per_reg:
            invalid_per_reg[rgi_reg] = set()

        if rgi_reg not in meta_per_reg:
            meta_per_reg[rgi_reg] = rgi_meta.loc[rgi_meta['O1Region'] == rgi_reg[-2:]]

        with xr.open_mfdataset(ff) as ds:
            ds_t = ds.isel(time=-1).volume.load()
        df_meta.loc[run_id, 'end_year'] = str(int(ds_t.time))
        missing_ids = ds_t.rgi_id[ds_t.isnull()].data
        perc = meta_per_reg[rgi_reg].loc[missing_ids]['Area'].sum() / meta_per_reg[rgi_reg]['Area'].sum() 
        df_meta.loc[run_id, 'perc_area_missing'] = perc
        invalid_per_reg[rgi_reg] = invalid_per_reg[rgi_reg].union(missing_ids)
        if f == allfiles[1000] or f==allfiles[2000]:
            print(f)
    df_meta['exp'] = exp
    df_meta['historical_future_option'] = historical_future_option
    df_meta['oggm_version'] = oggm_version
    df_meta.to_csv(f'{outputpath}/common_running_2100_2300/metadata.csv')
else:
    df_meta = pd.read_csv(f'{outputpath}/common_running_2100_2300/metadata.csv')

/home/www/oggm/oggm-standard-projections/oggm_v16/2023.3/CMIP6/2100/RGI12/run_hydro_w5e5_gcm_merged_CAMS-CSM1-0_ssp585_bc_2000_2019_Batch_0_1000.nc
/home/www/oggm/oggm-standard-projections/oggm_v16/2023.3/CMIP5/2100/RGI06/run_hydro_w5e5_gcm_merged_GISS-E2-R_rcp26_bc_2000_2019_Batch_0_1000.nc


In [6]:
dds = df_meta.sort_values('perc_area_missing', ascending=False)
dds[dds['perc_area_missing'] > 0.05].rgi_reg.unique()

array(['RGI12', 'RGI06', 'RGI08'], dtype=object)

In [7]:
dds

cmip   gcm_original  \
CMIP5:2100:CANESM2:rcp26:RGI12             CMIP5        CanESM2   
CMIP6:2100:NORESM2-MM:ssp126:RGI12         CMIP6     NorESM2-MM   
CMIP5:2100:IPSL-CM5A-LR:rcp26:RGI12        CMIP5   IPSL-CM5A-LR   
CMIP5:2100:MPI-ESM-LR:rcp26:RGI12          CMIP5     MPI-ESM-LR   
CMIP5:2100:CANESM2:rcp45:RGI12             CMIP5        CanESM2   
...                                          ...            ...   
CMIP6:2300:ACCESS-ESM1-5:ssp585:RGI07      CMIP6  ACCESS-ESM1-5   
CMIP6:2300:IPSL-CM6A-LR:ssp126:RGI07       CMIP6   IPSL-CM6A-LR   
CMIP6:2300:CANESM5:ssp126:RGI07            CMIP6        CanESM5   
CMIP6:2300:IPSL-CM6A-LR:ssp534-over:RGI07  CMIP6   IPSL-CM6A-LR   
CMIP6:2100:NORESM2-MM:ssp245:RGI07         CMIP6     NorESM2-MM   

                                                     gcm     scenario rgi_reg  \
CMIP5:2100:CANESM2:rcp26:RGI12                   CANESM2        rcp26   RGI12   
CMIP6:2100:NORESM2-MM:ssp126:RGI12            NORESM2-MM       ssp126   RGI12   
CMIP5:2100:IPSL-CM5A-LR:rcp26:RGI12         IPSL-CM5A-LR        rcp26   RGI12   
CMIP5:2100:MPI-ESM-LR:rcp26:RGI12             MPI-ESM-LR        rcp26   RGI12   
CMIP5:2100:CANESM2:rcp45:RGI12                   CANESM2        rcp45   RGI12   
...                                                  ...          ...     ...   
CMIP6:2300:ACCESS-ESM1-5:ssp585:RGI07      ACCESS-ESM1-5       ssp585   RGI07   
CMIP6:2300:IPSL-CM6A-LR:ssp126:RGI07        IPSL-CM6A-LR       ssp126   RGI07   
CMIP6:2300:CANESM5:ssp126:RGI07                  CANESM5       ssp126   RGI07   
CMIP6:2300:IPSL-CM6A-LR:ssp534-over:RGI07   IPSL-CM6A-LR  ssp534-over   RGI07   
CMIP6:2100:NORESM2-MM:ssp245:RGI07            NORESM2-MM       ssp245   RGI07   

                                          end_year bias_correction  \
CMIP5:2100:CANESM2:rcp26:RGI12                2100       2000-2019   
CMIP6:2100:NORESM2-MM:ssp126:RGI12            2100       2000-2019   
CMIP5:2100:IPSL-CM5A-LR:rcp26:RGI12           2100       2000-2019   
CMIP5:2100:MPI-ESM-LR:rcp26:RGI12             2100       2000-2019   
CMIP5:2100:CANESM2:rcp45:RGI12                2100       2000-2019   
...                                            ...             ...   
CMIP6:2300:ACCESS-ESM1-5:ssp585:RGI07         2300       2000-2019   
CMIP6:2300:IPSL-CM6A-LR:ssp126:RGI07          2300       2000-2019   
CMIP6:2300:CANESM5:ssp126:RGI07               2300       2000-2019   
CMIP6:2300:IPSL-CM6A-LR:ssp534-over:RGI07     2300       2000-2019   
CMIP6:2100:NORESM2-MM:ssp245:RGI07            2100       2000-2019   

                                           perc_area_missing  \
CMIP5:2100:CANESM2:rcp26:RGI12                      0.121243   
CMIP6:2100:NORESM2-MM:ssp126:RGI12                  0.120892   
CMIP5:2100:IPSL-CM5A-LR:rcp26:RGI12                 0.119703   
CMIP5:2100:MPI-ESM-LR:rcp26:RGI12                   0.119397   
CMIP5:2100:CANESM2:rcp45:RGI12                      0.119350   
...                                                      ...   
CMIP6:2300:ACCESS-ESM1-5:ssp585:RGI07               0.000000   
CMIP6:2300:IPSL-CM6A-LR:ssp126:RGI07                0.000000   
CMIP6:2300:CANESM5:ssp126:RGI07                     0.000000   
CMIP6:2300:IPSL-CM6A-LR:ssp534-over:RGI07           0.000000   
CMIP6:2100:NORESM2-MM:ssp245:RGI07                  0.000000   

                                                                                       fpath  \
CMIP5:2100:CANESM2:rcp26:RGI12             /home/www/oggm/oggm-standard-projections/oggm_...   
CMIP6:2100:NORESM2-MM:ssp126:RGI12         /home/www/oggm/oggm-standard-projections/oggm_...   
CMIP5:2100:IPSL-CM5A-LR:rcp26:RGI12        /home/www/oggm/oggm-standard-projections/oggm_...   
CMIP5:2100:MPI-ESM-LR:rcp26:RGI12          /home/www/oggm/oggm-standard-projections/oggm_...   
CMIP5:2100:CANESM2:rcp45:RGI12             /home/www/oggm/oggm-standard-projections/oggm_...   
...                                                                             

In [8]:
for k, v in invalid_per_reg.items():
    invalid_per_reg[k] = list(v)
with open(f'{outputpath}/common_running_2100_2300/rgi_ids_missing.json', 'w') as f:
    json.dump(invalid_per_reg, f)

In [9]:
# these are now the common running glaciers over all scenarios and GCMs and all cmip variants going until 2100 or until 2300
odf = pd.DataFrame()
for rgi_reg, missing_ids in invalid_per_reg.items():
    odf.loc[rgi_reg, 'n_glaciers'] = len(meta_per_reg[rgi_reg])
    odf.loc[rgi_reg, 'n_missing_glaciers'] = len(missing_ids)
    odf.loc[rgi_reg, 'rgi_area_km2'] = meta_per_reg[rgi_reg]['Area'].sum() 
    odf.loc[rgi_reg, 'missing_area_km2'] = meta_per_reg[rgi_reg].loc[missing_ids]['Area'].sum()
    odf.loc[rgi_reg, 'missing_area_perc'] = odf.loc[rgi_reg, 'missing_area_km2'] / odf.loc[rgi_reg, 'rgi_area_km2']
odf[['n_glaciers', 'n_missing_glaciers']] = odf[['n_glaciers', 'n_missing_glaciers']].astype(int)
odf = odf.sort_values(by='missing_area_perc', ascending=False)
odf

n_glaciers  n_missing_glaciers  rgi_area_km2  missing_area_km2  \
RGI12        1888                 409      1306.992           160.691   
RGI06         568                 420     11059.700           963.393   
RGI08        3417                1152      2949.103           237.021   
RGI10        5151                 243      2410.051            94.172   
RGI05       19306                1662     89717.066          1467.785   
RGI07        1615                  48     33958.934           238.343   
RGI13       54429                 862     49303.415           311.203   
RGI14       27988                 501     33568.298           102.782   
RGI19        2752                 633    132867.219           377.671   
RGI04        7415                 313     40888.228           100.893   
RGI17       15908                 132     29429.080            58.999   
RGI03        4556                 126    105110.642           201.883   
RGI02       18855                 461     14524.224            18.732   
RGI15       13119                  58     14734.204            14.869   
RGI09        1069                   9     51591.600            40.895   
RGI11        3927                  36      2092.146             1.327   
RGI16        2939                  12      2341.036             0.710   
RGI01       27108                  73     86725.053            14.635   
RGI18        3537                   1      1161.801             0.052   

       missing_area_perc  
RGI12           0.122947  
RGI06           0.087108  
RGI08           0.080371  
RGI10           0.039075  
RGI05           0.016360  
RGI07           0.007019  
RGI13           0.006312  
RGI14           0.003062  
RGI19           0.002842  
RGI04           0.002468  
RGI17           0.002005  
RGI03           0.001921  
RGI02           0.001290  
RGI15           0.001009  
RGI09           0.000793  
RGI11           0.000634  
RGI16           0.000303  
RGI01           0.000169  
RGI18           0.000045

In [10]:
odf.to_csv(f'{outputpath}/common_running_2100_2300/missing_region_overview.csv')
odf.to_html(f'{outputpath}/common_running_2100_2300/missing_region_overview.html')

### A. 2: GCMs - aggregation

In [11]:
base_dir =f'{outputpath}/common_running_2100_2300'
df_meta = pd.read_csv(f'{base_dir}/metadata.csv', index_col = 0)
import json
with open(f'{base_dir}/rgi_ids_missing.json', 'r') as f:
    invalid_per_reg = json.load(f)

In [12]:
for cmip in df_meta.cmip.unique():
    #print(cmip, flush=True)
    if cmip == 'CMIP6':
        end_years = [2100, 2300]
    else:
        end_years = [2100]
    for end_year in end_years:
        for rgi_reg in sorted(df_meta.rgi_reg.unique()): #progressbar.progressbar(
            for scenario in sorted(df_meta.loc[(df_meta.cmip==cmip) & (df_meta.end_year == end_year)].scenario.unique()):
                df_meta_s = df_meta.loc[(df_meta.end_year == end_year) & (df_meta.cmip == cmip) & (df_meta.rgi_reg == rgi_reg) & (df_meta.scenario == scenario)]
                odf_v = pd.DataFrame()
                odf_a = pd.DataFrame()
                ods = []
                gcms = []
                for i, s in df_meta_s.iterrows():
                    with xr.open_mfdataset(s.fpath) as ds:
                        ds = ds[['volume', 'area']].load().isel(rgi_id=~ds.rgi_id.isin(invalid_per_reg[s.rgi_reg]))
                        #ds = ds[['volume', 'area', 'length']].load().isel(rgi_id=~ds.rgi_id.isin(invalid_per_reg[s.rgi_reg]))
                        odf_v[s.gcm] = ds.volume.sum(dim='rgi_id').to_series()
                        odf_a[s.gcm] = ds.area.sum(dim='rgi_id').to_series()
                        ods.append(ds)
                        gcms.append(s.gcm)
                ods = xr.concat(ods, 'gcm')

                # we don't like to have these kind of summary statistics as the user does not know
                # whether the different projections are gaussian distributed ... 
                #ods.mean(dim='gcm').to_netcdf(f'output_1.6.1_2023.3/{exp}/{rgi_reg}/all_gcm_avg_{scenario}.nc')
                #ods.std(dim='gcm').to_netcdf(f'output_1.6.1_2023.3/{exp}/{rgi_reg}/all_gcm_std_{scenario}.nc')
                odir =  f'{base_dir}/volume/{cmip}/{end_year}/{rgi_reg}/'
                mkdir(odir)
                odf_v.to_csv(odir + f'{scenario}.csv')
                odir = f'{base_dir}/area/{cmip}/{end_year}/{rgi_reg}/'
                mkdir(odir)
                odf_a.to_csv(odir + f'{scenario}.csv')

In [14]:
# globally: sum over all RGI regions
for var in ['volume', 'area']:
    for cmip in df_meta.cmip.unique():
        if cmip == 'CMIP6':
            end_years = [2100, 2300]
        else:
            end_years = [2100]
        for end_year in end_years:
            odir = f'{base_dir}/{var}/{cmip}/{end_year}/global/'
            mkdir(odir)
            for scenario in sorted(df_meta.loc[(df_meta.cmip==cmip) & (df_meta.end_year == end_year)].scenario.unique()):
                odf = 0
                for rgi_reg in sorted(df_meta.rgi_reg.unique()):
                    idir = f'{base_dir}/{var}/{cmip}/{end_year}/{rgi_reg}/'
                    df = pd.read_csv(idir + f'/{scenario}.csv', index_col=0)
                    odf += df
                odf.to_csv(odir + f'/{scenario}.csv')

## B. Common running glacier aggregations for all projections until 2100, i.e. `common_running_2100`

### B. 1: GCMs - missing glaciers overview  

In [15]:
allfiles = []
for root, dirs, files in os.walk(dirpath):
    if not '/2300/' in root:
        for file in files:
            if (file.endswith(".nc")) and ('run_hydro' in file) and (historical_future_option in file) and ('_0_1000' in file) and ('basin' not in file):
                 allfiles.append(os.path.join(root, file))

df_meta = pd.DataFrame()
invalid_per_reg = {}  
meta_per_reg = {}

In [16]:
# this takes very long !!!
invalid_per_reg = {}  
meta_per_reg = {}
for f in allfiles: #progressbar.progressbar(allfiles):
    ename = f.replace(dirpath, '')
    ss = ename.split('/')
    cmip = ss[0]
    endyr = ss[1]
    rgi_reg = ss[2]
    gcm_original = ss[3].split('_')[5] 
    gcm = gcm_original.upper()
    if 'ISIMIP3b' in cmip:
        ssp = ss[3].split('_')[7]
        bc = 'ISIMIP3b:1979-2014'
    else:
        ssp = ss[3].split('_')[6]
        bc = '2000-2019'

    end0,end1 = ss[3].split('_')[-2:]
    end = end0+'_'+end1
    ff = f[:-len(end)]+'*'

    # exp is the same everywhere, so we don't need it as an id... 
    run_id = f'{cmip}:{endyr}:{gcm}:{ssp}:{rgi_reg}'
    df_meta.loc[run_id, 'cmip'] = cmip
    df_meta.loc[run_id, 'gcm_original'] = gcm_original # some have lower and upper cases in the raw output files
    df_meta.loc[run_id, 'gcm'] = gcm  # for the aggregated files, all GCMs are in upper cases
    df_meta.loc[run_id, 'scenario'] = ssp
    df_meta.loc[run_id, 'rgi_reg'] = rgi_reg
    df_meta.loc[run_id, 'end_year'] = 0  # will be updated later
    df_meta.loc[run_id, 'bias_correction'] = bc

    df_meta.loc[run_id, 'perc_area_missing'] = 0
    
    df_meta.loc[run_id, 'fpath'] = ff
    
    if rgi_reg not in invalid_per_reg:
        invalid_per_reg[rgi_reg] = set()
    
    if rgi_reg not in meta_per_reg:
        meta_per_reg[rgi_reg] = rgi_meta.loc[rgi_meta['O1Region'] == rgi_reg[-2:]]
    
    with xr.open_mfdataset(ff) as ds:
        ds_t = ds.isel(time=-1).volume.load()
    df_meta.loc[run_id, 'end_year'] = str(int(ds_t.time))
    missing_ids = ds_t.rgi_id[ds_t.isnull()].data
    perc = meta_per_reg[rgi_reg].loc[missing_ids]['Area'].sum() / meta_per_reg[rgi_reg]['Area'].sum() 
    df_meta.loc[run_id, 'perc_area_missing'] = perc
    invalid_per_reg[rgi_reg] = invalid_per_reg[rgi_reg].union(missing_ids)
    if f == allfiles[100]:
        print(f)
df_meta['exp'] = exp
df_meta['historical_future_option'] = historical_future_option
df_meta['oggm_version'] = oggm_version
df_meta.to_csv(f'{outputpath}/common_running_2100/metadata.csv')

/home/www/oggm/oggm-standard-projections/oggm_v16/2023.3/ISIMIP3b_CMIP6/2100/RGI12/run_hydro_w5e5_gcm_merged_gfdl-esm4_r1i1p1f1_ssp370_Batch_0_1000.nc


In [17]:
dds = df_meta.sort_values('perc_area_missing', ascending=False)
dds[dds['perc_area_missing'] > 0.05].rgi_reg.unique()

array(['RGI12'], dtype=object)

In [18]:
dds

cmip  gcm_original           gcm  \
CMIP5:2100:CANESM2:rcp26:RGI12            CMIP5       CanESM2       CANESM2   
CMIP6:2100:NORESM2-MM:ssp126:RGI12        CMIP6    NorESM2-MM    NORESM2-MM   
CMIP5:2100:IPSL-CM5A-LR:rcp26:RGI12       CMIP5  IPSL-CM5A-LR  IPSL-CM5A-LR   
CMIP5:2100:MPI-ESM-LR:rcp26:RGI12         CMIP5    MPI-ESM-LR    MPI-ESM-LR   
CMIP5:2100:CANESM2:rcp45:RGI12            CMIP5       CanESM2       CANESM2   
...                                         ...           ...           ...   
CMIP6:2100:CESM2-WACCM:ssp534-over:RGI07  CMIP6   CESM2-WACCM   CESM2-WACCM   
CMIP6:2100:IPSL-CM6A-LR:ssp585:RGI07      CMIP6  IPSL-CM6A-LR  IPSL-CM6A-LR   
CMIP6:2100:CANESM5:ssp534-over:RGI07      CMIP6       CanESM5       CANESM5   
CMIP6:2100:MRI-ESM2-0:ssp434:RGI07        CMIP6    MRI-ESM2-0    MRI-ESM2-0   
CMIP5:2100:CANESM2:rcp85:RGI07            CMIP5       CanESM2       CANESM2   

                                             scenario rgi_reg end_year  \
CMIP5:2100:CANESM2:rcp26:RGI12                  rcp26   RGI12     2100   
CMIP6:2100:NORESM2-MM:ssp126:RGI12             ssp126   RGI12     2100   
CMIP5:2100:IPSL-CM5A-LR:rcp26:RGI12             rcp26   RGI12     2100   
CMIP5:2100:MPI-ESM-LR:rcp26:RGI12               rcp26   RGI12     2100   
CMIP5:2100:CANESM2:rcp45:RGI12                  rcp45   RGI12     2100   
...                                               ...     ...      ...   
CMIP6:2100:CESM2-WACCM:ssp534-over:RGI07  ssp534-over   RGI07     2100   
CMIP6:2100:IPSL-CM6A-LR:ssp585:RGI07           ssp585   RGI07     2100   
CMIP6:2100:CANESM5:ssp534-over:RGI07      ssp534-over   RGI07     2100   
CMIP6:2100:MRI-ESM2-0:ssp434:RGI07             ssp434   RGI07     2100   
CMIP5:2100:CANESM2:rcp85:RGI07                  rcp85   RGI07     2100   

                                         bias_correction  perc_area_missing  \
CMIP5:2100:CANESM2:rcp26:RGI12                 2000-2019           0.121243   
CMIP6:2100:NORESM2-MM:ssp126:RGI12             2000-2019           0.120892   
CMIP5:2100:IPSL-CM5A-LR:rcp26:RGI12            2000-2019           0.119703   
CMIP5:2100:MPI-ESM-LR:rcp26:RGI12              2000-2019           0.119397   
CMIP5:2100:CANESM2:rcp45:RGI12                 2000-2019           0.119350   
...                                                  ...                ...   
CMIP6:2100:CESM2-WACCM:ssp534-over:RGI07       2000-2019           0.000000   
CMIP6:2100:IPSL-CM6A-LR:ssp585:RGI07           2000-2019           0.000000   
CMIP6:2100:CANESM5:ssp534-over:RGI07           2000-2019           0.000000   
CMIP6:2100:MRI-ESM2-0:ssp434:RGI07             2000-2019           0.000000   
CMIP5:2100:CANESM2:rcp85:RGI07                 2000-2019           0.000000   

                                                                                      fpath  \
CMIP5:2100:CANESM2:rcp26:RGI12            /home/www/oggm/oggm-standard-projections/oggm_...   
CMIP6:2100:NORESM2-MM:ssp126:RGI12        /home/www/oggm/oggm-standard-projections/oggm_...   
CMIP5:2100:IPSL-CM5A-LR:rcp26:RGI12       /home/www/oggm/oggm-standard-projections/oggm_...   
CMIP5:2100:MPI-ESM-LR:rcp26:RGI12         /home/www/oggm/oggm-standard-projections/oggm_...   
CMIP5:2100:CANESM2:rcp45:RGI12            /home/www/oggm/oggm-standard-projections/oggm_...   
...                                                                                     ...   
CMIP6:2100:CESM2-WACCM:ssp534-over:RGI07  /home/www/oggm/oggm-standard-projections/oggm_...   
CMIP6:2100:IPSL-CM6A-LR:ssp585:RGI07      /home/www/oggm/oggm-standard-projections/oggm_...   
CMIP6:2100:CANESM5:ssp534-over:RGI07      /home/www/oggm/oggm-standard-projections/oggm_...   
CMIP6:2100:MRI-ESM2-0:ssp434:RGI07        /home/www/oggm/oggm-standard-projections/oggm_...   
CMIP5:2100:CANESM2:rcp85:RGI07            /home/www/oggm/oggm-standard-projections/oggm_...   

                                                        exp  \
CMIP5:2100:CANESM2:rcp26:RGI12            match_geod

In [19]:
for k, v in invalid_per_reg.items():
    invalid_per_reg[k] = list(v)
with open(f'{outputpath}/common_running_2100/rgi_ids_missing.json', 'w') as f:
    json.dump(invalid_per_reg, f)

# these are now the common running glaciers over all scenarios and GCMs and all cmip variants going until 2100 or until 2300
odf = pd.DataFrame()
for rgi_reg, missing_ids in invalid_per_reg.items():
    odf.loc[rgi_reg, 'n_glaciers'] = len(meta_per_reg[rgi_reg])
    odf.loc[rgi_reg, 'n_missing_glaciers'] = len(missing_ids)
    odf.loc[rgi_reg, 'rgi_area_km2'] = meta_per_reg[rgi_reg]['Area'].sum() 
    odf.loc[rgi_reg, 'missing_area_km2'] = meta_per_reg[rgi_reg].loc[missing_ids]['Area'].sum()
    odf.loc[rgi_reg, 'missing_area_perc'] = odf.loc[rgi_reg, 'missing_area_km2'] / odf.loc[rgi_reg, 'rgi_area_km2']
odf[['n_glaciers', 'n_missing_glaciers']] = odf[['n_glaciers', 'n_missing_glaciers']].astype(int)
odf = odf.sort_values(by='missing_area_perc', ascending=False)
odf

n_glaciers  n_missing_glaciers  rgi_area_km2  missing_area_km2  \
RGI12        1888                 409      1306.992           160.691   
RGI10        5151                 168      2410.051            89.351   
RGI19        2752                 598    132867.219           301.246   
RGI13       54429                 345     49303.415            92.609   
RGI17       15908                  94     29429.080            49.960   
RGI03        4556                  77    105110.642           176.943   
RGI05       19306                 344     89717.066            88.617   
RGI15       13119                  41     14734.204            11.674   
RGI14       27988                 136     33568.298            23.149   
RGI11        3927                  31      2092.146             1.074   
RGI09        1069                   4     51591.600            23.984   
RGI08        3417                  17      2949.103             1.180   
RGI16        2939                  12      2341.036             0.710   
RGI02       18855                  96     14524.224             4.392   
RGI06         568                  11     11059.700             2.301   
RGI04        7415                  35     40888.228             6.180   
RGI01       27108                  51     86725.053            11.696   
RGI18        3537                   1      1161.801             0.052   
RGI07        1615                   0     33958.934             0.000   

       missing_area_perc  
RGI12           0.122947  
RGI10           0.037074  
RGI19           0.002267  
RGI13           0.001878  
RGI17           0.001698  
RGI03           0.001683  
RGI05           0.000988  
RGI15           0.000792  
RGI14           0.000690  
RGI11           0.000513  
RGI09           0.000465  
RGI08           0.000400  
RGI16           0.000303  
RGI02           0.000302  
RGI06           0.000208  
RGI04           0.000151  
RGI01           0.000135  
RGI18           0.000045  
RGI07           0.000000

In [20]:
odf.to_csv(f'{outputpath}/common_running_2100/missing_region_overview.csv')
odf.to_html(f'{outputpath}/common_running_2100/missing_region_overview.html')

### B. 2: GCMs - aggregation

In [21]:
base_dir =f'{outputpath}/common_running_2100'
df_meta = pd.read_csv(f'{base_dir}/metadata.csv', index_col = 0)
with open(f'{base_dir}/rgi_ids_missing.json', 'r') as f:
    invalid_per_reg = json.load(f)

In [ ]:
for cmip in df_meta.cmip.unique():
    print(cmip, flush=True)
    if cmip == 'CMIP6':
        end_years = [2100]
    else:
        end_years = [2100]
    for end_year in end_years:
        for rgi_reg in sorted(df_meta.rgi_reg.unique()): #progressbar.progressbar(
            for scenario in sorted(df_meta.loc[(df_meta.cmip==cmip) & (df_meta.end_year == end_year)].scenario.unique()):
                df_meta_s = df_meta.loc[(df_meta.end_year == end_year) & (df_meta.cmip == cmip) & (df_meta.rgi_reg == rgi_reg) & (df_meta.scenario == scenario)]
                odf_v = pd.DataFrame()
                odf_a = pd.DataFrame()
                ods = []
                gcms = []
                for i, s in df_meta_s.iterrows():
                    with xr.open_mfdataset(s.fpath) as ds:
                        ds = ds[['volume', 'area']].load().isel(rgi_id=~ds.rgi_id.isin(invalid_per_reg[s.rgi_reg]))
                        #ds = ds[['volume', 'area', 'length']].load().isel(rgi_id=~ds.rgi_id.isin(invalid_per_reg[s.rgi_reg]))
                        odf_v[s.gcm] = ds.volume.sum(dim='rgi_id').to_series()
                        odf_a[s.gcm] = ds.area.sum(dim='rgi_id').to_series()
                        ods.append(ds)
                        gcms.append(s.gcm)
                ods = xr.concat(ods, 'gcm')

                # I don't like to have these kind of summary statistics as the user does not know whether the different projections are gaussian distributed .. 
                #ods.mean(dim='gcm').to_netcdf(f'output_1.6.1_2023.3/{exp}/{rgi_reg}/all_gcm_avg_{scenario}.nc')
                #ods.std(dim='gcm').to_netcdf(f'output_1.6.1_2023.3/{exp}/{rgi_reg}/all_gcm_std_{scenario}.nc')
                odir =  f'{base_dir}/volume/{cmip}/{end_year}/{rgi_reg}/'
                mkdir(odir)
                odf_v.to_csv(odir + f'{scenario}.csv')
                odir = f'{base_dir}/area/{cmip}/{end_year}/{rgi_reg}/'
                mkdir(odir)
                odf_a.to_csv(odir + f'{scenario}.csv')

ISIMIP3b_CMIP6
CMIP6
CMIP5


In [ ]:
# globally: sum over all RGI regions
for var in ['volume', 'area']:
    for cmip in df_meta.cmip.unique():
        odir = f'{base_dir}/{var}/{cmip}/{end_year}/global/'
        mkdir(odir)
        for scenario in sorted(df_meta.loc[(df_meta.cmip==cmip) & (df_meta.end_year == end_year)].scenario.unique()):
            odf = 0
            for rgi_reg in sorted(df_meta.rgi_reg.unique()):
                idir = f'{base_dir}/{var}/{cmip}/{end_year}/{rgi_reg}/'
                df = pd.read_csv(idir + f'/{scenario}.csv', index_col=0)
                odf += df
            odf.to_csv(odir + f'/{scenario}.csv')

### C: common running with Rounce et al. (2023)

In [6]:
base_dir =f'/home/www/oggm/oggm-standard-projections/analysis_notebooks/1.6.1/common_running_rounce_et_al_2023'
df_meta = pd.read_csv(f'{outputpath}/common_running_2100/metadata.csv', index_col = 0)
import json
# this file has been created here: /home/www/oggm/oggm-standard-projections/oggm-standard-projections-csv-files/notebooks/analyse_csv_files_1.6.1.ipynb
with open(f'/home/www/oggm/oggm-standard-projections/analysis_notebooks/1.6.1/common_running_rounce_et_al_2023/rgi_ids_missing_rounce_et_al_or_oggm.json', 'r') as f:
    invalid_per_reg = json.load(f)

In [8]:
for cmip in ['CMIP6']:
    print(cmip, flush=True)
    if cmip == 'CMIP6':
        end_years = [2100]
    else:
        end_years = [2100]
    for end_year in end_years:
        for rgi_reg in sorted(df_meta.rgi_reg.unique()): #progressbar.progressbar(
            for scenario in ['ssp126', 'ssp245','ssp370', 'ssp585']:
                df_meta_s = df_meta.loc[(df_meta.end_year == end_year) & (df_meta.cmip == cmip) & (df_meta.rgi_reg == rgi_reg) & (df_meta.scenario == scenario)]
                odf_v = pd.DataFrame()
                odf_a = pd.DataFrame()
                ods = []
                gcms = []
                for i, s in df_meta_s.iterrows():
                    with xr.open_mfdataset(s.fpath) as ds:
                        ds = ds[['volume']].load().isel(rgi_id=~ds.rgi_id.isin(invalid_per_reg[s.rgi_reg]))
                        #ds = ds[['volume', 'area', 'length']].load().isel(rgi_id=~ds.rgi_id.isin(invalid_per_reg[s.rgi_reg]))
                        odf_v[s.gcm] = ds.volume.sum(dim='rgi_id').to_series()
                        ods.append(ds)
                        gcms.append(s.gcm)
                ods = xr.concat(ods, 'gcm')

                # I don't like to have these kind of summary statistics as the user does not know whether the different projections are gaussian distributed .. 
                #ods.mean(dim='gcm').to_netcdf(f'output_1.6.1_2023.3/{exp}/{rgi_reg}/all_gcm_avg_{scenario}.nc')
                #ods.std(dim='gcm').to_netcdf(f'output_1.6.1_2023.3/{exp}/{rgi_reg}/all_gcm_std_{scenario}.nc')
                odir =  f'{base_dir}/volume/{cmip}/{end_year}/{rgi_reg}/'
                mkdir(odir)
                odf_v.to_csv(odir + f'{scenario}.csv')


CMIP6


In [9]:
# globally: sum over all RGI regions
for var in ['volume']:
    for cmip in ['CMIP6']:
        if cmip == 'CMIP6':
            end_years = [2100]
        else:
            end_years = [2100]
        for end_year in end_years:
            odir = f'{base_dir}/{var}/{cmip}/{end_year}/global/'
            mkdir(odir)
            for scenario in ['ssp126', 'ssp245','ssp370', 'ssp585']:
                odf = 0
                for rgi_reg in sorted(df_meta.rgi_reg.unique()):
                    idir = f'{base_dir}/{var}/{cmip}/{end_year}/{rgi_reg}/'
                    df = pd.read_csv(idir + f'{scenario}.csv', index_col=0)
                    odf += df
                odf.to_csv(odir + f'{scenario}.csv')

In [10]:
df_meta

cmip   gcm_original  \
ISIMIP3b_CMIP6:2100:MRI-ESM2-0:ssp585:RGI11   ISIMIP3b_CMIP6     mri-esm2-0   
ISIMIP3b_CMIP6:2100:UKESM1-0-LL:ssp585:RGI11  ISIMIP3b_CMIP6    ukesm1-0-ll   
ISIMIP3b_CMIP6:2100:MRI-ESM2-0:ssp126:RGI11   ISIMIP3b_CMIP6     mri-esm2-0   
ISIMIP3b_CMIP6:2100:UKESM1-0-LL:ssp370:RGI11  ISIMIP3b_CMIP6    ukesm1-0-ll   
ISIMIP3b_CMIP6:2100:MRI-ESM2-0:ssp370:RGI11   ISIMIP3b_CMIP6     mri-esm2-0   
...                                                      ...            ...   
CMIP5:2100:GFDL-CM3:rcp45:RGI04                        CMIP5       GFDL-CM3   
CMIP5:2100:CSIRO-MK3-6-0:rcp26:RGI04                   CMIP5  CSIRO-Mk3-6-0   
CMIP5:2100:IPSL-CM5A-LR:rcp45:RGI04                    CMIP5   IPSL-CM5A-LR   
CMIP5:2100:GFDL-CM3:rcp85:RGI04                        CMIP5       GFDL-CM3   
CMIP5:2100:IPSL-CM5A-LR:rcp60:RGI04                    CMIP5   IPSL-CM5A-LR   

                                                        gcm scenario rgi_reg  \
ISIMIP3b_CMIP6:2100:MRI-ESM2-0:ssp585:RGI11      MRI-ESM2-0   ssp585   RGI11   
ISIMIP3b_CMIP6:2100:UKESM1-0-LL:ssp585:RGI11    UKESM1-0-LL   ssp585   RGI11   
ISIMIP3b_CMIP6:2100:MRI-ESM2-0:ssp126:RGI11      MRI-ESM2-0   ssp126   RGI11   
ISIMIP3b_CMIP6:2100:UKESM1-0-LL:ssp370:RGI11    UKESM1-0-LL   ssp370   RGI11   
ISIMIP3b_CMIP6:2100:MRI-ESM2-0:ssp370:RGI11      MRI-ESM2-0   ssp370   RGI11   
...                                                     ...      ...     ...   
CMIP5:2100:GFDL-CM3:rcp45:RGI04                    GFDL-CM3    rcp45   RGI04   
CMIP5:2100:CSIRO-MK3-6-0:rcp26:RGI04          CSIRO-MK3-6-0    rcp26   RGI04   
CMIP5:2100:IPSL-CM5A-LR:rcp45:RGI04            IPSL-CM5A-LR    rcp45   RGI04   
CMIP5:2100:GFDL-CM3:rcp85:RGI04                    GFDL-CM3    rcp85   RGI04   
CMIP5:2100:IPSL-CM5A-LR:rcp60:RGI04            IPSL-CM5A-LR    rcp60   RGI04   

                                              end_year     bias_correction  \
ISIMIP3b_CMIP6:2100:MRI-ESM2-0:ssp585:RGI11       2100  ISIMIP3b:1979-2014   
ISIMIP3b_CMIP6:2100:UKESM1-0-LL:ssp585:RGI11      2100  ISIMIP3b:1979-2014   
ISIMIP3b_CMIP6:2100:MRI-ESM2-0:ssp126:RGI11       2100  ISIMIP3b:1979-2014   
ISIMIP3b_CMIP6:2100:UKESM1-0-LL:ssp370:RGI11      2100  ISIMIP3b:1979-2014   
ISIMIP3b_CMIP6:2100:MRI-ESM2-0:ssp370:RGI11       2100  ISIMIP3b:1979-2014   
...                                                ...                 ...   
CMIP5:2100:GFDL-CM3:rcp45:RGI04                   2100           2000-2019   
CMIP5:2100:CSIRO-MK3-6-0:rcp26:RGI04              2100           2000-2019   
CMIP5:2100:IPSL-CM5A-LR:rcp45:RGI04               2100           2000-2019   
CMIP5:2100:GFDL-CM3:rcp85:RGI04                   2100           2000-2019   
CMIP5:2100:IPSL-CM5A-LR:rcp60:RGI04               2100           2000-2019   

                                              perc_area_missing  \
ISIMIP3b_CMIP6:2100:MRI-ESM2-0:ssp585:RGI11            0.000495   
ISIMIP3b_CMIP6:2100:UKESM1-0-LL:ssp585:RGI11           0.000495   
ISIMIP3b_CMIP6:2100:MRI-ESM2-0:ssp126:RGI11            0.000495   
ISIMIP3b_CMIP6:2100:UKESM1-0-LL:ssp370:RGI11           0.000495   
ISIMIP3b_CMIP6:2100:MRI-ESM2-0:ssp370:RGI11            0.000495   
...                                                         ...   
CMIP5:2100:GFDL-CM3:rcp45:RGI04                        0.000053   
CMIP5:2100:CSIRO-MK3-6-0:rcp26:RGI04                   0.000073   
CMIP5:2100:IPSL-CM5A-LR:rcp45:RGI04                    0.000063   
CMIP5:2100:GFDL-CM3:rcp85:RGI04                        0.000053   
CMIP5:2100:IPSL-CM5A-LR:rcp60:RGI04                    0.000057   

                                                                                          fpath  \
ISIMIP3b_CMIP6:2100:MRI-ESM2-0:ssp585:RGI11   /home/www/oggm/oggm-standard-projections/oggm_...   
ISIMIP3b_CMIP6:2100:UKESM1-0-LL:ssp585:RGI11  /home/www/oggm/oggm-standard-projections/oggm_...   
ISIMIP3b_CMIP6:2100:MRI-ESM2-0:ssp126:RGI11   /home/www/oggm/oggm-standard-projections/oggm_...   
ISIMIP3b

In [37]:
odf_2100= pd.read_csv(f'{outputpath}/common_running_2100/missing_region_overview.csv', index_col=0)
odf_2100.loc['Global'] = odf_2100.sum(axis=0)
# need to recompute the percentage
odf_2100['missing_area_perc'] = 100*odf_2100['missing_area_km2'] / odf_2100['rgi_area_km2']
odf_2100['missing_glaciers_perc'] = 100*odf_2100['n_missing_glaciers'] / odf_2100['n_glaciers']
odf_2100 = odf_2100.sort_index()

odf_2300= pd.read_csv(f'{outputpath}/common_running_2100_2300/missing_region_overview.csv', index_col=0)
odf_2300.loc['Global'] = odf_2300.sum(axis=0)
# need to recompute the percentage
odf_2300['missing_area_perc'] = 100*odf_2300['missing_area_km2'] / odf_2300['rgi_area_km2']
odf_2300['missing_glaciers_perc'] = 100*odf_2300['n_missing_glaciers'] / odf_2300['n_glaciers']
odf_2300 = odf_2300.sort_index()

In [38]:
import seaborn as sns

In [54]:
odf_area = odf_2100[['missing_area_perc']]
odf_area.index.name = 'Region'
odf_area.columns.name = 'Option of common running glaciers'
odf_area = odf_area.rename(columns={'missing_area_perc':'common_running_2100'})
odf_area['common_running_2100_2300'] = odf_2300['missing_area_perc']

odf_glac = odf_2100[['missing_glaciers_perc']]
odf_glac.index.name = 'Region'
odf_glac.columns.name = 'Option of common running glaciers'
odf_glac = odf_glac.rename(columns={'missing_glaciers_perc':'common_running_2100'})
odf_glac['common_running_2100_2300'] = odf_2300['missing_glaciers_perc']

In [56]:
plt.rc('font', size=16)
fig, axs = plt.subplots(1,2,figsize=(30,12))
ax = axs[0]
sns.heatmap(odf_area.astype(float), 
            ax=ax,
            annot=True, fmt=".2f",cmap="crest_r",linewidth=.5,
            cbar_kws={'label':'Missing glacier area (in %)'})
ax.set_title(f'Relative percentage of missing glacier area\nwhere at least one GCM or Scenario could not be run over the entire time period', fontsize=22)


ax = axs[1]
sns.heatmap(odf_glac.astype(float), annot=True, fmt=".1f",cmap="Reds",linewidth=.5, ax=ax,
            cbar_kws={'label':'Missing glaciers (in %)'})
ax.set_title(f'Relative percentage of missing glaciers\nwhere at least one GCM or Scenario could not be run over the entire time period', fontsize=22);
plt.savefig('missing_glacier_area_stats.png')